In [1]:
import oscilloscope
app = oscilloscope.oscilloscope()

In [2]:
app.display()

Loading BokehJS ...

In [3]:
from IPython.lib import backgroundjobs as bg
jobs = bg.BackgroundJobManager()

jobs.new('app.run()')

Starting job # 0 in a separate thread.


<BackgroundJob #0: app.run()>

In [ ]:
app.run()

In [ ]:
jobs.status()